### Add Policies to the Execution Role 
* In this sample code, we are going to use several AWS services. Therefore we have to add policies to the notebook execution role. 
* Regarding to role and policy, please refer to documents [1](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles.html) and [2](https://docs.aws.amazon.com/IAM/latest/UserGuide/access.html)



In [ ]:
import boto3
from sagemaker import get_execution_role

role_name = get_execution_role().split('/')[-1]

iam = boto3.client("iam")

print(role_name)
policy_arns = [
               "arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess",
               "arn:aws:iam::aws:policy/AmazonTextractFullAccess"
]

for p in policy_arns: 
    iam.attach_role_policy(
        RoleName = role_name,
        PolicyArn = p
    )



### Alternate Docker Storage Location 

* docker overlay directory usually will occupy large amount of disk space, change the location to EBS volume 

In [3]:
%%writefile daemon.json
{
    "runtimes": {
        "nvidia": {
            "path": "nvidia-container-runtime",
            "runtimeArgs": []
        }
    },
    "default-shm-size": "4096M"
}


Overwriting daemon.json


In [4]:
%%bash 
sudo service docker stop
sudo cp daemon.json /etc/docker/
mkdir ~/SageMaker/docker_disk
sudo mv /var/lib/docker ~/SageMaker/docker_disk/
sudo ln -s  ~/SageMaker/docker_disk/docker/ /var/lib/
sudo service docker start

Stopping docker: [  OK  ]
Starting docker:	.[  OK  ]


mkdir: cannot create directory ‘/home/ec2-user/SageMaker/docker_disk’: File exists
mv: ‘/var/lib/docker’ and ‘/home/ec2-user/SageMaker/docker_disk/docker’ are the same file
ln: failed to create symbolic link ‘/var/lib/docker’: File exists


In [ ]:
%%bash 
cd ~/SageMaker
git clone https://github.com/aws-samples/amazon-textract-code-samples.git 
wget -O Mmdetection.zip https://tinyurl.com/yfp7z4n6
wget -O icdar_table_cells_dataset.zip https://tinyurl.com/yftec3qv
unzip Mmdetection.zip
unzip icdar_table_cells_dataset.zip 
cp Mmdetection/new_chunk_cascade_mask_rcnn_hrnetv2p_w32_20e/epoch_36.pth CascadeTabNet/